# CAD integration


Steps:

1) Create the CAD files
2) Create a mesh ([SALOME](https://www.salome-platform.org/), [GMSH](https://gmsh.info/)...)
3) Convert the mesh to XDMF (using [meshio](https://github.com/nschloe/meshio))

In [1]:
import meshio


def convert_med_to_xdmf(
    medfilename,
    cell_file="mesh_domains.xdmf",
    facet_file="mesh_boundaries.xdmf",
    cell_type="tetra",
    facet_type="triangle",
):
    """Converts a MED mesh to XDMF
    Args:
        medfilename (_type_): _description_
        cell_file (str, optional): _description_. Defaults to "mesh_domains.xdmf".
        facet_file (str, optional): _description_. Defaults to "mesh_boundaries.xdmf".
        cell_type (str, optional): _description_. Defaults to "tetra".
        facet_type (str, optional): _description_. Defaults to "triangle".
    Returns:
        dict, dict: the correspondance dict, the cell types
    """
    msh = meshio.read(medfilename)

    correspondance_dict = msh.cell_tags

    cell_data_types = msh.cell_data_dict["cell_tags"].keys()

    for mesh_block in msh.cells:
        if mesh_block.type == cell_type:

            meshio.write_points_cells(
                cell_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][cell_type]]},
            )
        elif mesh_block.type == facet_type:
            meshio.write_points_cells(
                facet_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][facet_type]]},
            )

    return correspondance_dict, cell_data_types

In [2]:
correspondance_dict, cell_data_types = convert_med_to_xdmf("task7/mesh.med", cell_file="task7/mesh_domains.xdmf", facet_file="task7/mesh_boundaries.xdmf")

print(correspondance_dict)

{-6: ['tungsten'], -7: ['cu'], -8: ['cucrzr'], -9: ['top_surface'], -10: ['cooling_surface'], -11: ['poloidal_gap'], -12: ['toroidal_gap'], -13: ['bottom']}


In [3]:
import festim as F

my_mesh = F.MeshFromXDMF(volume_file="task7/mesh_domains.xdmf", boundary_file="task7/mesh_boundaries.xdmf")

Succesfully load mesh with 106966 cells
